In [2]:
import numpy as np
import gtsam
import matplotlib
import matplotlib.pyplot as plt

from minslam.data_loader import TartanAirLoader, plot_trajectory
from minslam.frontend import Frontend
from minslam.params import Params
from minslam.backend import Backend
from minslam.camera import PinholeCamera

fig = plt.figure()

def run_frontend_once(frontend):
    pose = dataset.read_current_ground_truth()
    while not frontend.keyframe_selection(pose):
        if not dataset.load_next_frame():
            break
        pose = dataset.read_current_ground_truth()
    color, depth = dataset.read_current_rgbd()
    frontend.add_keyframe(pose, color, depth, dataset.curr_index)
    print(f'--- Added keyframe {frontend.frame_id} (seq id: {dataset.curr_index}) ---')
    more_points_n = params['frontend']['feature']['number']
    frontend.extract_features(more_points_n, append_mode=False)
    print('extracting features:', len(frontend.curr_frame.keypoints), f'(expected {more_points_n})')
    if frontend.frame_id > 0:
        frontend.match_features()
        print('matching features:', len(frontend.curr_frame.matches))
        frontend.eliminate_outliers()
    frontend.assign_global_id()
    return frontend.curr_frame

# load the dataset
params = Params('./params/tartanair.yaml')
frontend = Frontend(params)
backend = Backend(params)
dataset = TartanAirLoader('./data/P006/')
start_index = 0
dataset.set_curr_index(start_index)
n_keyframes = 10

# read the ground truth and odometry
dataset.load_ground_truth()
gt_poses = dataset.gt
# odom_poses = dataset.add_noise(gt_poses, [1e-4, 3e-4], [1e-3, 1e-3], seed=100, start=start_index)
dataset.set_odometry(gt_poses)

FileNotFoundError: Parameter file not found

<Figure size 640x480 with 0 Axes>

In [ ]:
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
im = ax.imshow(np.zeros([480, 640, 3]))     # A tuple unpacking to unpack the only plot

# run the whole pipeline once
def run_once(frame_num):
    pose = dataset.read_current_odometry()
    while not frontend.keyframe_selection(pose):
        if not dataset.load_next_frame():
            break
        pose = dataset.read_current_odometry()
    color, depth = dataset.read_current_rgbd()
    frontend.add_keyframe(pose, color, depth)
    print(f'--- Added keyframe {frontend.frame_id} (seq id: {dataset.curr_index}) ---')
    if frontend.frame_id > 0:
        frontend.match_features()
        frontend.eliminate_outliers()
    more_points_n = params['frontend']['feature']['number']-len(frontend.curr_frame.points)
    print('current point number:', len(frontend.curr_frame.points))
    if more_points_n>0:
        print('extracting more features:', more_points_n)
        frontend.extract_features(more_points_n, append_mode=True)
    frontend.assign_global_id()
    img = frontend.plot_matches(with_global_id=True)
    im.set_array(img)
    return [im]

# generate tracking animation
anim = FuncAnimation(fig, run_once, frames=100)
anim.save('recitation_tracking.mp4', writer='ffmpeg', fps=10)